In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os 
import glob
from skimage.feature import hog
import time
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
#sklearn


In [2]:
"""
데이터 적재 
"""

'\n데이터 적재 \n'

In [67]:
car_images = []
noncar_images = []
for root, dirs, files in os.walk('./data/vehicles/'):
    for file in files:
        if file.endswith(".png"):
            car_images.append(os.path.join(root, file))
            
for root, dirs, files in os.walk('./data/non-vehicles/'):
    for file in files:
        if file.endswith(".png"):
            noncar_images.append(os.path.join(root, file))

In [4]:
car_images

['./data/vehicles/GTI_Far\\image0000.png',
 './data/vehicles/GTI_Far\\image0001.png',
 './data/vehicles/GTI_Far\\image0002.png',
 './data/vehicles/GTI_Far\\image0003.png',
 './data/vehicles/GTI_Far\\image0004.png',
 './data/vehicles/GTI_Far\\image0005.png',
 './data/vehicles/GTI_Far\\image0006.png',
 './data/vehicles/GTI_Far\\image0007.png',
 './data/vehicles/GTI_Far\\image0008.png',
 './data/vehicles/GTI_Far\\image0009.png',
 './data/vehicles/GTI_Far\\image0010.png',
 './data/vehicles/GTI_Far\\image0014.png',
 './data/vehicles/GTI_Far\\image0015.png',
 './data/vehicles/GTI_Far\\image0016.png',
 './data/vehicles/GTI_Far\\image0018.png',
 './data/vehicles/GTI_Far\\image0019.png',
 './data/vehicles/GTI_Far\\image0021.png',
 './data/vehicles/GTI_Far\\image0022.png',
 './data/vehicles/GTI_Far\\image0023.png',
 './data/vehicles/GTI_Far\\image0024.png',
 './data/vehicles/GTI_Far\\image0027.png',
 './data/vehicles/GTI_Far\\image0029.png',
 './data/vehicles/GTI_Far\\image0030.png',
 './data/ve

In [5]:
noncar_images

['./data/non-vehicles/Extras\\extra1.png',
 './data/non-vehicles/Extras\\extra10.png',
 './data/non-vehicles/Extras\\extra100.png',
 './data/non-vehicles/Extras\\extra1000.png',
 './data/non-vehicles/Extras\\extra1001.png',
 './data/non-vehicles/Extras\\extra1002.png',
 './data/non-vehicles/Extras\\extra1003.png',
 './data/non-vehicles/Extras\\extra1004.png',
 './data/non-vehicles/Extras\\extra1005.png',
 './data/non-vehicles/Extras\\extra1006.png',
 './data/non-vehicles/Extras\\extra1007.png',
 './data/non-vehicles/Extras\\extra1008.png',
 './data/non-vehicles/Extras\\extra1009.png',
 './data/non-vehicles/Extras\\extra101.png',
 './data/non-vehicles/Extras\\extra1010.png',
 './data/non-vehicles/Extras\\extra1011.png',
 './data/non-vehicles/Extras\\extra1012.png',
 './data/non-vehicles/Extras\\extra1013.png',
 './data/non-vehicles/Extras\\extra1014.png',
 './data/non-vehicles/Extras\\extra1015.png',
 './data/non-vehicles/Extras\\extra102.png',
 './data/non-vehicles/Extras\\extra103.png

In [6]:
#테스트용 100개 이미지
car_images = car_images
noncar_images = noncar_images
imgs = car_images + noncar_images

In [7]:
"""
이미지 리사이즈
"""
def img_resize(img):
    src = cv2.imread(img)
    src_resize = cv2.resize(src, (32,32)) # fx: 가로 사이즈의 배수
    resize_feature = src_resize.flatten()

    return resize_feature

In [8]:
resize_features_list = []
for file in car_images:
    resize_feature = img_resize(file)
    resize_features_list.append(img_resize)

    
for file in noncar_images:
    resize_feature = img_resize(file)
    resize_features_list.append(img_resize)

In [9]:
resize_feature

array([ 51,  58,  42, ..., 115, 129, 123], dtype=uint8)

In [10]:
resize_feature.shape

(3072,)

In [11]:
"""
hog 알고리즘 
"""
def hog_img(img):
    hog_img = mpimg.imread(img)
    gray_img = cv2.cvtColor(hog_img, cv2.COLOR_BGR2GRAY)
    
    hog_feature, hog_image = hog(gray_img, orientations=9, pixels_per_cell=(4,4), 
                                     cells_per_block=(2, 2), block_norm='L2-Hys', 
                                     transform_sqrt=False, visualize = True, 
                                     feature_vector=False)
    
    hog_feature = hog_feature.flatten()
        
    return hog_feature

In [12]:
hog_features_list = []
for file in car_images:
    hog_feature = hog_img(file)
    hog_features_list.append(hog_img)

    
for file in noncar_images:
    hog_feature = hog_img(file)
    hog_features_list.append(hog_img)

In [13]:
hog_feature

array([0.19099373, 0.        , 0.        , ..., 0.        , 0.        ,
       0.        ])

In [14]:
hog_feature.shape

(8100,)

In [15]:
"""
히스토그램 분석
"""
def hist_img(img):
    
    src = cv2.imread(img)
    hist1_b = cv2.calcHist(images = [src], 
                         channels = [0], 
                         mask = None, 
                         histSize = [32], 
                         ranges = [0, 255])

    hist2_g = cv2.calcHist(images = [src], 
                         channels = [1], 
                         mask = None, 
                         histSize = [32], 
                         ranges = [0, 255])

    hist3_r = cv2.calcHist(images = [src], 
                         channels = [2], 
                         mask = None, 
                         histSize = [32], 
                         ranges = [0, 255])

    hist1_b_f = hist1_b.flatten()
    hist2_g_f = hist2_g.flatten()
    hist3_r_f = hist3_r.flatten()

    hist_feature = np.concatenate([hist1_b_f, hist2_g_f, hist3_r_f]) # 특성 합치는 함수

    return hist_feature

In [16]:
hist_features_list = []
for file in car_images:
    hist_feature = hist_img(file)
    hist_features_list.append(hist_img)

    
for file in noncar_images:
    hist_feature = hist_img(file)
    hist_features_list.append(hist_img)

In [17]:
hist_feature

array([  0.,   0.,   0.,   0.,   1.,  87., 448., 365., 287., 518., 181.,
       134.,  86.,  75., 162., 221., 185., 441., 549., 113.,  38.,  38.,
        35.,  40.,  49.,  35.,   8.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   5., 125., 386., 312., 546., 310., 170.,
       117., 111.,  73.,  74., 215., 165., 163., 329., 580., 169.,  44.,
        31.,  38.,  36.,  53.,  35.,   9.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   7., 122., 296., 320., 515., 319., 123., 156., 113.,
       122.,  86., 118., 190., 145., 132., 224., 397., 432.,  73.,  37.,
        33.,  47.,  44.,  39.,   5.,   1.,   0.,   0.], dtype=float32)

In [18]:
hist_feature.shape

(96,)

In [19]:
hist_features_list

[<function __main__.hist_img(img)>,
 <function __main__.hist_img(img)>,
 <function __main__.hist_img(img)>,
 <function __main__.hist_img(img)>,
 <function __main__.hist_img(img)>,
 <function __main__.hist_img(img)>,
 <function __main__.hist_img(img)>,
 <function __main__.hist_img(img)>,
 <function __main__.hist_img(img)>,
 <function __main__.hist_img(img)>,
 <function __main__.hist_img(img)>,
 <function __main__.hist_img(img)>,
 <function __main__.hist_img(img)>,
 <function __main__.hist_img(img)>,
 <function __main__.hist_img(img)>,
 <function __main__.hist_img(img)>,
 <function __main__.hist_img(img)>,
 <function __main__.hist_img(img)>,
 <function __main__.hist_img(img)>,
 <function __main__.hist_img(img)>,
 <function __main__.hist_img(img)>,
 <function __main__.hist_img(img)>,
 <function __main__.hist_img(img)>,
 <function __main__.hist_img(img)>,
 <function __main__.hist_img(img)>,
 <function __main__.hist_img(img)>,
 <function __main__.hist_img(img)>,
 <function __main__.hist_img

In [20]:
"""
레이블 - 머신러닝 정답 
"""
label_list = []
for i in range(len(car_images)):
    answer=1
    label_list.append(answer)

    
for i in range(len(noncar_images)):
    notanswer=0
    label_list.append(notanswer)

In [21]:
labeled = np.array(label_list)

In [22]:
resize_features_list

[<function __main__.img_resize(img)>,
 <function __main__.img_resize(img)>,
 <function __main__.img_resize(img)>,
 <function __main__.img_resize(img)>,
 <function __main__.img_resize(img)>,
 <function __main__.img_resize(img)>,
 <function __main__.img_resize(img)>,
 <function __main__.img_resize(img)>,
 <function __main__.img_resize(img)>,
 <function __main__.img_resize(img)>,
 <function __main__.img_resize(img)>,
 <function __main__.img_resize(img)>,
 <function __main__.img_resize(img)>,
 <function __main__.img_resize(img)>,
 <function __main__.img_resize(img)>,
 <function __main__.img_resize(img)>,
 <function __main__.img_resize(img)>,
 <function __main__.img_resize(img)>,
 <function __main__.img_resize(img)>,
 <function __main__.img_resize(img)>,
 <function __main__.img_resize(img)>,
 <function __main__.img_resize(img)>,
 <function __main__.img_resize(img)>,
 <function __main__.img_resize(img)>,
 <function __main__.img_resize(img)>,
 <function __main__.img_resize(img)>,
 <function _

In [23]:
resize_feature

array([ 51,  58,  42, ..., 115, 129, 123], dtype=uint8)

In [24]:
resize_feature.shape

(3072,)

In [25]:
hog_feature

array([0.19099373, 0.        , 0.        , ..., 0.        , 0.        ,
       0.        ])

In [26]:
hog_feature.shape

(8100,)

In [27]:
hist_feature

array([  0.,   0.,   0.,   0.,   1.,  87., 448., 365., 287., 518., 181.,
       134.,  86.,  75., 162., 221., 185., 441., 549., 113.,  38.,  38.,
        35.,  40.,  49.,  35.,   8.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   5., 125., 386., 312., 546., 310., 170.,
       117., 111.,  73.,  74., 215., 165., 163., 329., 580., 169.,  44.,
        31.,  38.,  36.,  53.,  35.,   9.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   7., 122., 296., 320., 515., 319., 123., 156., 113.,
       122.,  86., 118., 190., 145., 132., 224., 397., 432.,  73.,  37.,
        33.,  47.,  44.,  39.,   5.,   1.,   0.,   0.], dtype=float32)

In [28]:
hist_feature.shape

(96,)

In [29]:
feature_list = []
for file in imgs:
    feature1 = img_resize(file)
    feature2 = hog_img(file)
    feature3 = hist_img(file)
    feature = np.concatenate((feature1, feature2, feature3))
    feature_list.append(feature)
X = np.array(feature_list)

In [30]:
X.shape

(17760, 11268)

In [31]:
X.dtype

dtype('float64')

In [32]:
X

array([[163., 168., 156., ...,   0.,   0.,   0.],
       [163., 168., 156., ...,   0.,   0.,   0.],
       [139., 152., 143., ...,   0.,   0.,   0.],
       ...,
       [ 48.,  62.,  55., ...,   0.,   0.,   0.],
       [ 57.,  71.,  65., ...,   0.,   0.,   0.],
       [ 51.,  58.,  42., ...,   1.,   0.,   0.]])

In [33]:
from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler()

In [34]:
X_scaled = std_scaler.fit_transform(X)

In [35]:
y= labeled

In [36]:
y.shape

(17760,)

In [37]:
"""
머신러닝 부분 - 
문제는 값이 -1~1 을 넘어선값
"""

'\n머신러닝 부분 - \n문제는 값이 -1~1 을 넘어선값\n'

In [38]:
from sklearn.model_selection import train_test_split

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y)

### 모델링

In [40]:
from sklearn.svm import LinearSVC

In [41]:
lin_SVC = LinearSVC()

In [42]:
lin_SVC.fit(X_train, y_train)

LinearSVC()

In [44]:
from sklearn.svm import SVC

In [45]:
svc = SVC()

In [46]:
svc.fit(X_train, y_train)

SVC()

In [48]:
from sklearn.ensemble import RandomForestClassifier

In [50]:
r_forest = RandomForestClassifier()

In [51]:
r_forest.fit(X_train, y_train)

RandomForestClassifier()

### 모델 평가

In [52]:
from sklearn.model_selection import cross_val_score

In [53]:
y_scores_lin_SVC = cross_val_score(lin_SVC, X_train, y_train, scoring="accuracy", cv=5, n_jobs=-1)

In [54]:
y_scores_lin_SVC

array([0.97522523, 0.97785285, 0.97672673, 0.97747748, 0.97897898])

In [55]:
y_scores_lin_SVC.mean()

0.9772522522522523

In [56]:
y_scores_SVC = cross_val_score(svc, X_train, y_train, scoring="accuracy", cv=5, n_jobs=-1)

In [57]:
y_scores_SVC

array([0.98761261, 0.99399399, 0.99174174, 0.99324324, 0.99174174])

In [58]:
y_scores_SVC.mean()

0.9916666666666666

In [59]:
y_scores_r_forest = cross_val_score(r_forest, X_train, y_train, scoring="accuracy", cv=5, n_jobs=-1)

In [60]:
y_scores_r_forest

array([0.98123123, 0.98460961, 0.98085586, 0.97635135, 0.98273273])

In [61]:
y_scores_r_forest.mean()

0.9811561561561563

### 예측

In [63]:
final_model = svc

In [64]:
y_pred = final_model.predict(X_test)

In [65]:
y_pred

array([1, 1, 0, ..., 0, 1, 0])

In [66]:
y_pred.shape

(4440,)

### 다음 수행 과제 ↓↓↓↓↓↓↓↓↓↓↓↓